In [1]:
import pandas as pd
import twint
import nest_asyncio # Nécessaire pour la librairie Twint
from unidecode import unidecode
from collections import Counter

import re
import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.fr.stop_words import STOP_WORDS

nest_asyncio.apply()

# Présentation du Projet

Dans le cadre du cours text mining, nous avons travaillé sur des données issues de Twitter concernant le projet avorté de la super ligue européenne. Ce projet est l'occasion d'aborder toutes les thématiques classique d'un projet de NLP.
Ainsi, celui-ci se divise en quatre partie :
- une première partie ou nous avons extrait les tweets qui concernaient notre sujet, puis effectuer du data cleaning pour filtrer les données qui n'apportaient pas d'informations supplémentaires.
- une seconde partie ou l'accent a été mis sur la visualisation. Quels sont les acteurs en jeu ? Quels personnalités ou clubs se sont distingués dans cet évènement ? Et dans quelle proportion ?
- une troisième partie ou on s'est attaché à modéliser nos données, et à faire des techniques de NLP, notamment concernant le topic modeling ou le clustering ainsi que via des modèles de deep learning utilisant la méthode du word embeddings 
- et une dernière partie, ou il s'est agi de créer une application streamlit permettant de mieux visualiser les résultats de nos modèles ainsi que de déployer notre application via le site [Herokuapp](https://streamlit-twitter-app.herokuapp.com/) grâce à la création également d'un répertoire sur le site GitHub.

## Téléchargement des données à partir du module Twint

Ainsi, pour effectuer notre recherche nous avons utilisé le hashtag superleague afin de récupérer les tweets qui nous intéressaient La librairie Twint nous évite de passer par l'API de Twitter, qui s'est révélée assez contraignante.

In [2]:
config = twint.Config()
config.Search = "#superleague superleague"
config.Lang = "fr"
config.Since = "2021-04-16 00:00:00"
config.Until = "2021-04-22 00:00:00"
config.Store_json = True
config.Output = "raw_tweets.json"

# La derniere ligne lance la recherche
#twint.run.Search(config)

### Pre-Processing

Malgré que l'option *Lang = 'fr'* ait été choisie, de nombreux tweets ne sont pas en français. Il convient donc des les filtrer.\
Du fait de la taille du data frame, on procède par *chunk* de 10000 lignes, on séléctionne les tweets en français puis nous créons un nouveau fichier csv.

In [3]:
def preprocess_tweets():
    chunks = pd.read_json('../Data_raw/raw_tweets.json', lines=True, chunksize = 10000)
    for chunk in chunks:
        result = chunk[chunk['language'] == 'fr']
        result.to_csv('../Data_clean/tweets_fr.csv', index=False, header=True, mode='a')

preprocess_tweets()

In [4]:
df = pd.read_csv('../Data_clean/tweets_fr.csv')
df.shape

(521394, 36)

### Suppression des NaN et des doublons

In [5]:
# La fonction preprocess_tweets ajoute un header pour chaque chunk, il convient donc de le retirer
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True, errors='coerce')
# Supprime les observations qui n'ont pas de date
df = df[~df['date'].isnull()] 

print(f"La table fait {df.shape[0]} lignes et {df.shape[1]} colonnes" )
print()
df.info()

La table fait 521052 lignes et 36 colonnes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 521052 entries, 0 to 521393
Data columns (total 36 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               521052 non-null  object        
 1   conversation_id  521052 non-null  object        
 2   created_at       521052 non-null  object        
 3   date             521052 non-null  datetime64[ns]
 4   time             521052 non-null  object        
 5   timezone         521052 non-null  object        
 6   user_id          521052 non-null  object        
 7   username         521052 non-null  object        
 8   name             521024 non-null  object        
 9   place            56 non-null      object        
 10  tweet            521052 non-null  object        
 11  language         521052 non-null  object        
 12  mentions         521052 non-null  object        
 13  urls             521052 non-nu

In [6]:
# Supprime les colonnes qui ne sont composées que de NaN.
df.dropna(axis=1, how='all',  inplace=True)

df.shape

(521052, 26)

In [7]:
# On supprime les tweets en double
df = df[~df.duplicated()]
df = df[~df['id'].duplicated()].reset_index()
df.shape

(37349, 27)

#### Data Cleaning

Utilisation de la liste des stop words issue de la librairie Spacy. Nous y avons ajouté quelques mots qui ne nous semblaient pas apporter de sens supplémentaires à notre projet. Les mots faisant référence à la superleague étant particulièrement redondant, car se trouvant forcément dans chaque tweet ont été supprimé. 

In [8]:
nlp = spacy.load("fr_core_news_md")

add_stop_words = ['europeansuperleague', "superleague", "superleagu","super", 'foot', 'football','super', "club",
                  'étaient', 'étais', 'était', 'étant', 'été', 'être', 'etre', 'etes', 'faire', 'aller', 'voir',
                  'ô',  'à', 'â', 'ça', 'ès','a', 'b','c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
                  'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'qu']

for word in add_stop_words:
    STOP_WORDS.add(word)

In [9]:
def clubs_signataires(x, clubs):
    """
    Création d'une fonction pour identifier les clubs  signataires, et faciliter le travail de 
    nos modèles de topic modeling.
    """
    string = ' '.join(['clubs_signataires' if word  in clubs else word for word in x.split() ])
    return string

signataires = ['arsenal', 'chelsea', 'chelseafc', 'chelseer', 'liverpool', 
                'manchestercity', 'mancity','manchester', 'manchesterunited', 'manutd', 'mufc',
               'tottenham hotspur', 'tottenham', 'spur', 'spurs',
                     'intermilan', 'juventus', 'juventu', 'juventusfc', 'juv', 'juve', 'turin', 
               'milan',  'acmilan','milanac', 'madrid', 'atletico', 'atleti', 'inter',
               'barcelona', 'barcelone', 'barca', 'fcbarca', 'fcbarcelone', 'fcbarcelona',
                'real', 'realmadrid', 'realmadridfrer']


def clubs_non_signataires(x, clubs):
    """
    Création d'une fonction pour identifier les clubs non signataires, et faciliter le travail de 
    nos modèles de topic modeling.
    """
    string = ' '.join(['clubs_non_signataires' if word  in clubs else word for word in x.split() ])
    return string

non_signataires = ['paris saint germain', 'psg', 'bayern munich', 'bayern', 'fcbayern',
                   'munich', 'borussia', 'dortmund']


def florentino(x, alias):
    """
    Création d'une fonction pour identifier les différents alias utilisés pour parler de Florentino Perez et
    permettre une meilleure identification de celui-ci
    """
    string = ' '.join(['florentino_perez' if word  in alias else word for word in x.split() ])
    return string

florentino_alias = ['perez', 'florentino', 'florentin', 'florentino_perer',
                    'florentinoperer', 'florentinoperez', 'per', 'perer']


def agnelli(x, alias):
    """
    Création d'une fonction pour identifier les différents alias utilisés pour parler de Andreas Agnelli et
    permettre une meilleure identification de celui-ci
    """
    string = ' '.join(['andrea_agnelli' if word  in alias else word for word in x.split() ])
    return string

agnelli_alias = ['andrea agnelli', 'andrea agneli', 'andrea', 'agnelli', 'agneli', 'andreer']



def superleague_out(x, alias):
    """
    Création d'une fonction pour identifier les différents alias utilisés pour exprimer le rejet du
    projet de superleague
    """
    string = ' '.join(['superleagueout' if word  in alias else word for word in x.split() ])
    return string

slogans_out = ['superleagueout', 'saynotoeuropeansuperleagu', 'saynotosuperleagu', 'saynotosuperleague',
      'notoeuropeansuperleagu','notoeuropeansuperleague', 'notosuperleagu', 'notosuperleague',
      'nonalasuperleagu', 'boycottsuperleagu', 'boycottsuperleague']


def suppression_alias(x, alias):
    """
    Suppresion de valeurs redondantes qui ont normalement déjà étaient prises en comptes.
    """
    string = ' '.join(['' if word  in alias else word for word in x.split() ])
    return string

sup_alias = ['paris saint germain', 'city', 'united', 'ac', 'fc', 'man'
                   'rt', 'fav']

In [10]:
def tweet_cleaner(pandasSeries):
    """
    Fonction ayant pour but de nettoyer nos tweets.
    Prend comme paramètre un pandas series.
    Opérations effectuées : mise au format string, minuscule, suppression des #, des mentions de compte,
    des URL, puis lemmatisation de nos données via Spacy, et suppression des stop words. Ensuite,
    traitement des données afin d'identifier les différents alias utilisés pour parler de clubs ou de personnalités.
    """
    
    print("#### Nettoyage en cours ####") 
    
    # Confirmation que chaque article est bien de type str
    pandasSeries = pandasSeries.apply(lambda x : str(x))
    
    # Passage en minuscule
    print("... Passage en minuscule") 
    pandasSeries = pandasSeries.apply(lambda x : x.lower())

    # Suppression des #
    print("... Suppression des #") 
    pandasSeries = pandasSeries.apply(lambda x :re.sub(r"#", '', x))
    
    # Suppression des noms de compte #
    print("... Suppression des @account") 
    pandasSeries = pandasSeries.apply(lambda x :re.sub(r"@\S+", '', x))
    
    # Suppression des url #
    print("... Suppression des url") 
    pandasSeries = pandasSeries.apply(lambda x :re.sub(r"http\S+", '', x))
    
    # Lemmatisation
    print("... Lemmatisation and Suppression des Stop words")
    
    pandasSeries = pd.Series((nlp.pipe(pandasSeries)))
    
    tweets_clean = []
    for tweet in pandasSeries:
        tokens_clean = []
        for token in tweet:
            if (token.lemma_ not in STOP_WORDS):
                tokens_clean.append((unidecode(token.lemma_)))
        tweets_clean.append(' '.join(tokens_clean))
    
    pandasSeries = pd.Series(tweets_clean)
    
    ## Suppression des caractères spéciaux et numériques
    print("... Suppression des caractères spéciaux et numériques") 
    pandasSeries = pandasSeries.apply(lambda x :re.sub(r"[^a-z]+", ' ', x))
    
    print("... Identifications des clubs non signataires")
    pandasSeries = pandasSeries.apply(lambda x : clubs_non_signataires(x, non_signataires))    
    
    print("... Identifications des clubs signataires")
    pandasSeries = pandasSeries.apply(lambda x : clubs_signataires(x, signataires))
    
    print("... Identifications des différents alias pour Florentino Perez")
    pandasSeries = pandasSeries.apply(lambda x : florentino(x, florentino_alias))
    
    print("... Identifications des différents alias pour Andrea Agnelli")
    pandasSeries = pandasSeries.apply(lambda x : agnelli(x, agnelli_alias))
    
    print("... Identifications des différents termes qualifiant le rejet du projet")
    pandasSeries = pandasSeries.apply(lambda x : superleague_out(x, slogans_out))
    
    print("... Suppression d'alias")
    pandasSeries = pandasSeries.apply(lambda x : suppression_alias(x, sup_alias))
    
    print("#### Nettoyage OK! ####")

    return pandasSeries

In [11]:
%%time 

df['tweet_clean'] = tweet_cleaner(df['tweet'])

#### Nettoyage en cours ####
... Passage en minuscule
... Suppression des #
... Suppression des @account
... Suppression des url
... Lemmatisation and Suppression des Stop words
... Suppression des caractères spéciaux et numériques
... Identifications des clubs non signataires
... Identifications des clubs signataires
... Identifications des différents alias pour Florentino Perez
... Identifications des différents alias pour Andrea Agnelli
... Identifications des différents termes qualifiant le rejet du projet
... Suppression d'alias
#### Nettoyage OK! ####
CPU times: user 1min 47s, sys: 1.83 s, total: 1min 48s
Wall time: 1min 49s


In [12]:
df[df['tweet_clean'].isna()][['tweet']]

,tweet


In [13]:
# Création d'une colonne pour identifier les jours du mois
df['day'] = df['date'].dt.day

In [14]:
cols_to_keep = ['date', 'time', 'tweet', 'tweet_clean', 'hashtags', 'likes_count', 'retweets_count', 'day']

tweets = df[cols_to_keep]
tweets.head()

,date,time,tweet,tweet_clean,hashtags,likes_count,retweets_count,day
0,2021-04-21,22:29:44,Jean Michel incisif face aux 12 frondeurs de l...,jean michel incisif face frondeur teamom jamais,"['superleague', 'teamom']",5,1,21
1,2021-04-21,22:29:01,Beppe #Marotta (AD Inter) lance un cri d’alarm...,beppe marotta ad clubs_signataires lancer cri ...,"['marotta', 'superleague']",56,18,21
2,2021-04-21,22:27:10,"Avec l'échec du projet de #SuperLeague, la mor...",echec projet mort visage olasm,"['superleague', 'olasm']",6,0,21
3,2021-04-21,22:26:00,Jean Michel trop heureux de la mort de la #Sup...,jean michel trop heureux mort teamom,"['superleague', 'teamom']",7,0,21
4,2021-04-21,22:25:48,3-1 pour la Juve contre Parme (2 Coupes UEFA) ...,clubs_signataires contre parme coupe uefa comp...,['superleague'],2,2,21


### Création du Fichier Csv Final

In [15]:
# Création d'un nouveau fichier csv qu'on utilisera par la suite
tweets.to_csv('../Data_clean/tweets_fr_clean.csv', index=False, header=True)